In [1]:
#take tags only from test5 database

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter


#df = pd.read_csv('train.csv')
test_data = pd.read_csv('test2_all.csv')

# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)

#take tags from the column 'photoTags'
tags = test_data.loc[:,'photoTags']

# join them to list
text = ' '.join(test_data['photoTags'])
text_list = text.split(' ')

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit_transform(tags)



print(len(vectorizer.vocabulary_))


# save the corpus into the file
import csv

dt = vectorizer.vocabulary_


d = pd.DataFrame.from_dict(dt, orient='index', dtype=None)

d.to_csv("corpus1.csv", encoding="utf-8", header=None)

#LOAD DATA
crp = pd.read_csv("corpus1.csv", header=None)
crp.to_csv("corpus1.csv", encoding="utf-8", header=['words','id'], index=None)

#LOAD DATA
crp = pd.read_csv("corpus1.csv")

#CLEAN DATA
crp['words'] = crp['words'].str.replace("\d+", '')
crp['words'] = crp['words'].str.replace("sec", '')
crp['words'] = crp['words'].str.replace("nabkv", '')
# delete values less then 3
crp =crp[crp['words'].str.len() > 3]
# delete geolat geolon
crp['words'] = crp['words'].str.replace("geolat", '')
crp['words'] = crp['words'].str.replace("geolon", '')
crp['words'] = crp['words'].str.replace("geotagged", '')

#train['words'] = train['words'].str.replace("^[0-9]*$", '')
#s = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s)
crp['words'].replace('', np.nan, inplace=True)
crp = crp.dropna(subset=['words'])

crp = crp[crp['words'].str.len() > 3]

# save data
crp.to_csv("corpus.csv", encoding="utf-8",index=None)

# load data again 
crp = pd.read_csv('corpus.csv')
#take tags from the column 'photoTags'
tags = crp.loc[:,'words']

# so we have features 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tf_matrix = tfidf.fit_transform(tags)

vocab = tfidf.vocabulary_



31180


In [2]:
# we have sparse matrix for 3 mln pictures 
# create KNN
from sklearn.neighbors import NearestNeighbors
train_data = pd.read_csv('dm_clean1.csv')
train_data_ = train_data.loc[:,'photoTags']
_vect = TfidfVectorizer(vocabulary = vocab)
train_c_matrix = _vect.fit_transform(train_data_)

#nbrs = NearestNeighbors(n_neighbors = 1, algorithm='ball_tree')
#nbrs.fit(train_c_matrix)

test_data = pd.read_csv('test2_all.csv')
# delete all data with empty tags from test data
#test_data.dropna(subset=['photoTags'], inplace=True)
test_data['photoTags'].replace( np.nan,'', inplace=True)
test_data_ = test_data.loc[:,'photoTags']
new_vect = TfidfVectorizer(vocabulary = vocab)
test_c_matrix = new_vect.fit_transform(test_data_)


In [3]:
# dimensionality reduction 
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=30, n_iter=7, random_state=42)
svd_test_matrix = svd.fit_transform(test_c_matrix)
svd_train_matrix = svd.fit_transform(train_c_matrix)

In [3]:
nbrs = NearestNeighbors(n_neighbors = 1, algorithm='ball_tree')
nbrs.fit(train_c_matrix)

C:\Apps\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:211: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=1, p=2, radius=1.0)

In [4]:
print(1)

1


In [ ]:
# BallTree for faster 
from sklearn.neighbors import KDTree
tree = KDTree(svd_train_matrix, leaf_size = 30, metric='manhattan')

In [ ]:
import datetime
predicted_lt = []
predicted_lng = []
#indexes = []
print(datetime.datetime.now())

for i in range(0,len(test_data_)):
    distances, indices = nbrs.kneighbors(test_c_matrix[[i]])

    #distances, indices = tree.query([svd_test_matrix[i]], k=1)
    ind = indices[0][0]
    #indexes.append(indices[0][0])
    if i%100==0:
        print(i)
        print(datetime.datetime.now())            
    #print('ind',ind)
    #print(distances)

    lt =train_data.loc[ind,'latitude']
    predicted_lt.append(lt)
    lng = train_data.loc[ind,'longitude']
    predicted_lng.append(lng)
    

    #df_lt = pd.DataFrame([lt], columns=['lt'])
    #df_ln = pd.DataFrame([lng], columns=['ln'])
    
    #data_lt = data_lt.append(df_lt)
    #data_ln = data_ln.append(df_ln)
    #print('test:', test_data.loc[i,'photoTags'])
    #print('train:', train_data.loc[ind,'photoTags'])
    #print('-----------------------------')
    

test_data['lt_predicted'] = predicted_lt
test_data['lng_predicted'] = predicted_lng

test_data.to_csv("predicted_3.csv", encoding="utf-8")

data_predicted = pd.read_csv('predicted_3.csv')

# let's calculate the error and save it in a new column 
lt_error = (data_predicted['lt_predicted'] - data_predicted['latitude'])**(2)
ln_error = (data_predicted['lng_predicted'] - data_predicted['longitude'])**(2)
er = (lt_error + ln_error)**(0.5)
data_predicted['error'] = er

# we will see radius of 0.001 km, 0.01 km, 1 km, 10 km , 100 km , 1000 km, 10000 km, 40000 km 

data_predicted.to_csv("predicted_3_error.csv", encoding="utf-8")

2018-05-16 11:07:35.117900
0
2018-05-16 11:07:35.667918
100
2018-05-16 11:08:13.029551
200
2018-05-16 11:08:50.560776
300
2018-05-16 11:09:27.697087
400
2018-05-16 11:10:04.542912
500
2018-05-16 11:10:41.549143
600
2018-05-16 11:11:18.540353
700
2018-05-16 11:11:55.861589
800
2018-05-16 11:12:33.070701
900
2018-05-16 11:13:09.991909
1000
2018-05-16 11:13:47.003120
1100
2018-05-16 11:14:23.806759
1200
2018-05-16 11:15:00.767970
1300
2018-05-16 11:15:37.679173
1400
2018-05-16 11:16:15.001407
1500
2018-05-16 11:16:52.002761
1600
2018-05-16 11:17:29.184209
1700
2018-05-16 11:18:06.225645
1800
2018-05-16 11:18:43.113282
1900
2018-05-16 11:19:20.424732
2000
2018-05-16 11:19:57.116157
2100
2018-05-16 11:20:34.129788
2200
2018-05-16 11:21:11.335361
2300
2018-05-16 11:21:47.993049
2400
2018-05-16 11:22:19.724468
2500
2018-05-16 11:22:54.556045
2600
2018-05-16 11:23:28.632928
2700
2018-05-16 11:24:01.579308
2800
2018-05-16 11:24:34.440774
2900
2018-05-16 11:25:07.482296
3000
2018-05-16 11:25:40.

In [10]:
print(len(svd_train_matrix[0]))

100
